<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Losses

Goal : have a table that compares the log likelihood of the models

Data : SVHN, CIFAR32 (MNIST Already Ran in Losses)

Models : SelfAttnLNP and ConvLNPXL and ConvCNPXL 

Loss : NLLLloss and CNPF

Runs : 5

In [ ]:
import os

os.chdir("..")

import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [2]:
args = get_exp_args("exp_hetero_lb001", is_load=False)
args.extend(get_exp_args("exp_hetero_lb01", is_load=False))
len(args)

60

In [3]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24,  
                           cpus_per_task=10, 
                           mem='32GB',
                           #partition="dev"
                          )

In [4]:
jobs = executor.map_array(Run(), args)

In [5]:
############################################################

In [19]:
jobs

[SlurmJob<job_id=27752115_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_5, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_6, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_7, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_8, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_9, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_10, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_11, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_12, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_13, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_14, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_15, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=27752115_16, task

In [18]:
for j in jobs:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2020-06-29 00:26:06,357) - Starting with JobEnvironment(job_id=27752115_0, hostname=learnfair0381, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-06-29 00:26:06,357) - Loading pickle: /private/home/yannd/projects/NPF/logs/27752115_0/27752115_0_submitted.pkl

--- Training mnist/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1    -1347.6249    -1076.3820     +  978.6752
      2    -2628.5719    -2929.8410     +  980.8677
      3    -2800.1590    -2956.6824     +  978.1809
      4    -2855.7729    -3023.6288     +  977.8276
      5    -2865.0348    -2912.8850        979.7794
      6    -2924.5687    -3145.8743     +  977.9739
      7    -2852.4419    -3079.3961        978.9351
      8    -3019.2620    -3131.4785        978.5914
      9    -2975.4362    -3161.5990     +  977.9415
     10    -3015.3565    -3094.7082        972.0495
  

In [14]:
for j in jobs:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
submitit ERROR (2020-06-26 18:03:48,685) - Submitted job triggered an exception
Traceback (most recent call last):
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/_submit.py", line 6, in <module>
    submitit_main()
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 186, in submitit_main
    process_job(args.folder)
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 179, in process_job
    raise error
  File "/private/home/yannd/.conda/envs/neuralproc/lib/python3.7/site-packages/submitit/submission.py", line 159, in process_job
    delayed = ut

In [9]:
for job in jobs:
    job.cancel()